# KeyBERT

In [4]:
from keybert import KeyBERT

kw_model = KeyBERT('bert-base-nli-mean-tokens')

/Users/jeon-yewon/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
doc = 'KeyBERT is a minimal and easy-to-use keyword extraction technique that leverages BERT embeddings to create keywords and keyphrases that are most similar to a document.'

keywords = kw_model.extract_keywords(doc)

print(keywords)

[('easy', 0.5055), ('keyphrases', 0.476), ('minimal', 0.4497), ('keyword', 0.3599), ('technique', 0.3567)]


In [7]:
# sentence transformers 활용

# SBERT(Sentence-BERT)

In [9]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
sentence1 = 'This is an example sentence'
sentence2 = 'This is a sample sentence'

embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)

In [12]:
cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

print('코사인 유사도 : ', cosine_similarity.item())

코사인 유사도 :  0.7881680130958557


In [13]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

In [16]:
n_gram_range = (3, 3) # trigram 사용
stop_words = 'english'

# CountVectorizer을 사용하던 n-gram을 쉽게 추출할 수 있다.
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수 : ', len(candidates))
print(candidates[:5])

trigram 개수 :  72
['algorithm analyzes training' 'algorithm correctly determine'
 'algorithm generalize training' 'allow algorithm correctly'
 'analyzes training data']


In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [21]:
# 코사인 유사도를 통해 해당 trigram들과 유사도가 높으면
# 문서를 대표할 수 있다는 의미를 나타낸다.
top_n = 5

dictances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in dictances.argsort()[0][-top_n:]]

In [23]:
print(keywords)

['training examples supervised', 'function labeled training', 'supervised learning machine', 'supervised learning example', 'supervised learning algorithm']


In [39]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # doc_embedding을 2차원 배열로 변환
    doc_embedding = np.array(doc_embedding).reshape(1, -1)
    
    # 문서와 각 후보 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)[0]

    # 각 후보 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings)

    # 상위 nr_candidates 개의 키워드들을 뽑는다.
    words_idx = list(distances.argsort()[-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 연관성이 낮은 키워드 찾기
    min_sim = np.inf  # 최소값 설정(np.inf : 무한대)
    candidate = None  # 키워드 저장 변수

    # itertools.combinations : 상위 top_n개의 조합
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])  # 각 조합에 대한 유사도 합산(자기 자신 제외)
        if sim < min_sim:  # 최솟값도 낮은 유사도
            candidate = combination  # 최적 후보로 등록
            min_sim = sim  # 최솟값 변경

    return [words_vals[idx] for idx in candidate]  # 리스트 내 데이터 뽑아서 반환

In [40]:
# 상위 10개의 키워드를 뽑고 이 중 연관성이 낮은 5개를 선택하겠다.
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['task learning function',
 'supervisory signal supervised',
 'labeled training data',
 'examples supervised learning',
 'supervised learning algorithm']

In [45]:
# Maximal Marginal Relevance : 텍스트 요약 처리(중복 최소화)
def max_mar_rel(doc_embedding, cadidate_embeddings, words, top_n, diversity):
    # 문서의 각 키워드들 간의 유사도
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 가장 높은 유사도를 가진 문서의 인덱스
    keywords_idx = [np.argmax(word_doc_similarity)]
    # 가장 높은 유사도를 제외한 인덱스들
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # top_n - 1 : 이미 가장 높은 유사도는 추출
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :] # 각 키워드들 간의 유사도
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1) # 가장 높은 유사도와 유사도 계산

        # MMR 점수 계산 (1 - diversity : 문서와의 관련성)
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)] # 최고 MMR

        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [46]:
max_mar_rel(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['supervised learning algorithm',
 'function labeled training',
 'supervised learning example',
 'supervisory signal supervised',
 'supervised learning machine']

In [48]:
from konlpy.tag import Okt

doc = """
         드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
         서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
         드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
         영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
         드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
         지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
         이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
         소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
         유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
         "향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
      """

In [49]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print(tokenized_doc[:10])
print(tokenized_nouns)

[('\n         ', 'Foreign'), ('드론', 'Noun'), ('활용', 'Noun'), ('범위', 'Noun'), ('도', 'Josa'), ('점차', 'Noun'), ('확대', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation')]
드론 활용 범위 점차 확대 최근 미세먼지 관리 드론 활용 서울시 미세먼지 계절 관리제 기간 지난달 개 월간 드론 측정 장치 달 미세먼지 집중 관리 실시 드론 산업 단지 사업 밀집 지역 미세먼지 배출 수치 점검 현장 모습 영상 영상 통해 미세먼지 방지 시설 제대로 가동 업체 무허가 시설 대한 단속 한층 전망 드론 활용 가장 적극 소방청 복합 재난 대응 차원 드론 관련 전문 인력 보강 어가 지난해 말 기준 소방청 보유 드론 총 드론 조종 자격증 소방대 경우 명 이 중 실기 평가 지도 자격증 드론 전문가 명도 배치 소방청 관계자 소방 드론 재난 현장 영상 정보 수집 산악 수난 사고 시 인명 수색 구조 활동 유독가스 폭발사고 시 대원 안전 확보 등 활용 며 향후 화재 진압 인명구조 등 드론 활용 위해 연구개발 고 말


In [50]:
n_gram_range = (2, 3) # bigram, trigram

count = CountVectorizer(ngram_range = n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('n-gram 개수 :', len(candidates))
print(candidates[:5])

n-gram 개수 : 222
['가동 업체' '가동 업체 무허가' '가장 적극' '가장 적극 소방청' '경우 실기']


In [51]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [54]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [55]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [56]:
keywords

['드론 산업', '드론 드론 조종', '실시 드론 산업', '관리 드론 활용', '미세먼지 관리 드론']

In [57]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['드론 산업 단지', '전망 드론 활용', '드론 산업', '관리 드론 활용', '미세먼지 관리 드론']

In [58]:
max_mar_rel(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['미세먼지 관리 드론', '실시 드론 산업', '관리 드론 활용', '월간 드론 측정', '전망 드론 활용']